In [2]:
import csv
from pathlib import Path

from base import BaseScript
from qa_frame.models.avro.schemas.video import Content
from qa_frame.consts.garm import GarmLabel
from qa_frame.interfaces.db.snowflake import SnowflakeDriver

In [34]:
class ReadPosts(BaseScript):
    name = "read_posts"
    def run(self):
        driver = SnowflakeDriver(self.snowflake_connection, self.config.interface.db.snowflake.environment)
        base = Path("ams_hpq_tiktok_labels.sql").read_text(encoding="utf-8")

        return {row["CONTENT_ID"][1:-1]: row for row in driver._read(base + """
                    , CONTENT_TIERS AS (
SELECT
  discovered.content_id AS content_id,
  discovered.impressions as impressions,
  IFNULL(discovered.inference_tier, 1) AS tier
FROM
  (
    SELECT
      content_id,
      max(impressions) as impressions,
      max(inference_tier) AS inference_tier
    FROM
      (
        SELECT
          content_id,
          adgroup_id,
          impressions,
          CASE WHEN percent_rank <= 0.00004 THEN 4 WHEN percent_rank <= 0.0002 THEN 3 WHEN percent_rank <= 0.08 THEN 2 -- Disable tier 1 as the campaign scanner is dumping all impressions to tier1. We do not want duplicates from two different sources (kafka, snowflake)
          -- ELSE 1
          END AS inference_tier
        FROM
          (
            SELECT
              *
            FROM
              (
                SELECT
                  content_id,
                  1 - percent_rank() OVER (
                    PARTITION BY adgroup_id
                    ORDER BY
                      impressions
                  ) AS percent_rank,
                  adgroup_id,
                  impressions
                FROM
                  (
                    SELECT
                      concat(page_id, '_', post_id) AS content_id,
                      ADSET_ID AS adgroup_id,
                      REPORT_DATE,
                      count(*) AS impressions
                    FROM
                      MEASUREMENT.META_CONTENT_ADJACENCY_DATA
                    GROUP BY
                      POST_ID,
                      PAGE_ID,
                      ADSET_ID,
                      REPORT_DATE
                  )
              )
          )
      )
    GROUP BY
      content_id
  ) discovered
)
        SELECT HPQ.*, CT.tier, CT.IMPRESSIONS
        FROM HPQ
        INNER JOIN CONTENT_TIERS CT
        ON CT.content_id = HPQ.content_id
        WHERE HPQ.content_id in (
            SELECT DISTINCT RECORD_CONTENT:payload:content_id
            FROM STAGING.DMDS_META_ADSET_IMPRESSIONS
            WHERE RECORD_CONTENT:payload:campaign_id IN ('23852701967600014')
        )
                        """)}

posts = ReadPosts().run()

{
  "asctime": "2023-02-15 15:04:43",
  "levelname": "INFO",
  "lineno": 54,
  "msg": "Initialized output directory",
  "output_path": "/Users/ryan.demarigny/PycharmProjects/qa-frame-notebooks/notebooks/output/read_posts"
}

{
  "asctime": "2023-02-15 15:04:44",
  "levelname": "INFO",
  "lineno": 79,
  "msg": "Connected to snowflake",
  "connection": {
    "host": "127.0.0.1",
    "port": "10017",
    "protocol": "http",
    "user": "ryandem",
    "account": "any",
    "database": "PROD",
    "warehouse": "ENGINEER_WH",
    "environment": "Environment.PROD"
  }
}



In [35]:
class WriteToCSV(BaseScript):
    name = "tiratat_cats_csv_generator"
    csv_name = "tiratat_cats_campaigns_moderators_decision.csv"
    folder = "campaign_3_moderators"
    campaign_name = "1P/3P internal house ads - Tiratat Cats - 3"

    def write_headers(self):
        with open(self.csv_name, "a") as csv_file:
            writer = csv.writer(csv_file)
            writer.writerow(["content_id", "models_decision", "campaign_name", "impressions", "tier"])

    def run(self):
        for content in self._read_json_files(self.folder, Content):
            row = [content.content_id]

            suitable = True
            if not content.derived:
                continue

            for tax_id in content.derived.taxonomy_scores:
                try:
                    label = GarmLabel.from_taxonomy_id(tax_id)
                except KeyError:
                    continue

                if label.risk_level not in ("MINIMAL", "UNMEASURABLE"):
                    if content.derived.taxonomy_scores[tax_id].score == 1:
                        suitable = False
                        break

            row.append({True: "suitable", False: "unsuitable"}[suitable])
            row.append(self.campaign_name)
            row.append(posts[content.content_id]["IMPRESSIONS"])
            row.append(posts[content.content_id]["TIER"])
            with open(self.csv_name, "a") as csv_file:
                writer = csv.writer(csv_file)
                writer.writerow(row)

WriteToCSV().run()

{
  "asctime": "2023-02-15 15:04:49",
  "levelname": "INFO",
  "lineno": 54,
  "msg": "Initialized output directory",
  "output_path": "/Users/ryan.demarigny/PycharmProjects/qa-frame-notebooks/notebooks/output/tiratat_cats_csv_generator"
}

{
  "asctime": "2023-02-15 15:04:49",
  "levelname": "INFO",
  "lineno": 152,
  "msg": "Reading file",
  "file_path": "/Users/ryan.demarigny/PycharmProjects/qa-frame-notebooks/notebooks/input/campaign_3_moderators/appen_standardized_job_content_8.json"
}

{
  "asctime": "2023-02-15 15:04:49",
  "levelname": "INFO",
  "lineno": 152,
  "msg": "Reading file",
  "file_path": "/Users/ryan.demarigny/PycharmProjects/qa-frame-notebooks/notebooks/input/campaign_3_moderators/appen_standardized_job_content_4.json"
}

{
  "asctime": "2023-02-15 15:04:49",
  "levelname": "INFO",
  "lineno": 152,
  "msg": "Reading file",
  "file_path": "/Users/ryan.demarigny/PycharmProjects/qa-frame-notebooks/notebooks/input/campaign_3_moderators/appen_standardized_job_content_5.

In [37]:
campaign_content = {"1P/3P internal house ads - Tiratat Cats - 1": set(), "1P/3P internal house ads - Tiratat Cats - 2": set(), "1P/3P internal house ads - Tiratat Cats - 3": set()}

campaign_lists = {"1P/3P internal house ads - Tiratat Cats - 1": [], "1P/3P internal house ads - Tiratat Cats - 2": [], "1P/3P internal house ads - Tiratat Cats - 3": []}


with open("tiratat_cats_campaigns_tiered_models_decision.csv", "r") as csv_file:
    reader = csv.reader(csv_file)
    suitable_count = 0
    unsuitable_count = 0
    next(reader)
    for row in reader:
        campaign_content[row[2]].add(row[0].split("_")[1])
        campaign_lists[row[2]].append(row[0])
        if row[1] == "suitable":
            suitable_count += 1
        else:
            unsuitable_count += 1

for campaign in campaign_content:
    print(len(campaign_content[campaign]))
    print(len(campaign_lists[campaign]))

print(suitable_count / (suitable_count + unsuitable_count))

291
291
193
193
1601
1601
0.6071942446043166
